In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 범죄 데이터 준비
df_crime = pd.read_excel("data-files/crime-in-seoul.xlsx", header=3)
df_crime2 = df_crime.iloc[:, -10:].copy()
df_crime2["자치구"] = df_crime.iloc[:, 1]
df_crime2

C:\Users\human\AppData\Roaming\Python\Python311\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,발생.55,검거.55,발생.56,검거.56,발생.57,검거.57,발생.58,검거.58,발생.59,검거.59,자치구
0,150,144,129,117,5764,5395,37038,21600,43132,37567,소계
1,5,6,5,7,239,1138,1169,953,1563,1381,종로구
2,3,3,6,5,212,144,1520,864,1607,1449,중구
3,3,4,3,3,245,219,978,561,1792,1481,용산구
4,2,1,2,2,119,85,893,472,1007,851,성동구
5,2,2,5,3,238,178,1616,830,1563,1320,광진구
6,5,6,4,3,146,116,1280,847,1522,1329,동대문구
7,6,6,2,2,169,134,1402,786,1745,1546,중랑구
8,1,1,1,1,141,112,984,683,1284,1161,성북구
9,7,7,1,1,141,102,917,730,1431,1302,강북구
